# 6. Język interpretowany AWK - część 2

Druga część z opisem i przykładami użycia `awk`.

## 6.1 Przekazywanie zmiennej do `awk`

AWK oferuje możliwość przekazania zmiennej zdefuniowanej w `bash`-u do wewnątrz komendy `awk`, aby jej użyć tamże. Należy do tego użyć opcji `-v` i wskazać nazwę zmiennej, której będziemy używać wewnątrz `awk` i do niej przypisać zmienną z `bash`-a.

In [54]:
nazwa_sys='Linux'    # zdefiniujmy zmienną znakową

In [55]:
awk -v name="$nazwa_sys" 'BEGIN{print "Mój system operacyjny to "name}'    # opcja -v przekazuje zmienną do awk

Mój system operacyjny to Linux


> W tym miejscu warto wspomnieć o specjalnym sposobie odnoszenia się do __zmiennej znakowej__ w `bash`-u. Otóż, można wypisać dowonly jej fragment, specyfikując:
- __offset__ : indeks od którego zacząć wypisywanie
- __lenght__ : liczba znaków do wypisania, zaczynając od indeksu określonego przez offset, jeśli ujemna - pominie wskazaną liczbę znakó od końca napisu
- `{zmiennaznakowa:offset:lenght}`

    Poniżej przykład:

In [80]:
echo ${nazwa_sys:2:3}     # wypisuje tylko znaki od 3 do 5 (indeks 2 oraz długość 3) ze słowa Linux

nux


In [91]:
echo ${nazwa_sys:0:-2}     # wypisuje od początku z wyjątkiem 2 ostatnich znaków

Lin


## 6.2 Wskazanie separatora do czytania pliku

Warto jest mieć możliwość wybrania znaku, który oddziela kolumny czytane przez `awk`. Oferuje to opcja `-F`. 
```bash
awk -F 'separator' '<warunek> {<polecenie>}'
```

Przykład:

In [95]:
head TIC12974182-comma.dat    # mamy plik z kolumnami oddzielonymi przecinkami

1354.1150770,-0.0655089,0.1957890
1354.1164659,-0.2563426,0.1957854
1354.1192437,0.0557094,0.1957512
1354.1206326,0.3493596,0.1958556
1354.1220215,-0.0107903,0.1958148
1354.1234104,0.0459730,0.1958106
1354.1247993,-0.1063045,0.1957953
1354.1261882,-0.1997741,0.1958022
1354.1275771,0.1834512,0.1958027
1354.1289660,0.3575382,0.1958367


In [106]:
awk -F ',' 'NR<5{print $3}' TIC12974182-comma.dat    # jako separator wskazany został przecinek

0.1957890
0.1957854
0.1957512
0.1958556


## 6.3 Wypisywanie formatowane

W `awk` można zdefiniować w jaki sposób chcemy wypisać nasze dane. Możemy wskazać __typ__ naszej zmiennej (m.in. __zmienna znakowa - string__, __liczba rzeczywista - float__, __całkowita - integer__) oraz na ilu znakach ma zostać wypisana oraz w przypadku liczby rzeczywistej, ile miejsc po przecinku ma być uwzględnione.

Służy do tego funkcja __printf__.<br>Przykład:

In [116]:
awk -F ',' 'NR<5{printf("%7.4f\n", $2)}' TIC12974182-comma.dat

-0.0655
-0.2563
 0.0557
 0.3494


W powyższym przykładzie wypisane zostały pierwsze 4 wartości z drugiej kolumny. Format wypisywania został określony dzięki temu zapisowi: `%11.8f`. Oznacza, to że liczby zostały wypisane na __7 miejscach__, z __4 miejscami po przecinku__. Ten sposób pozwala na eleganckie wypisywanie z wyrównaniem: 4 miejsca po przecinku, kropka, jedno zero i ewentualny znak (minus), dają razem 7 miejsc. Zobaczmy jak to będzie wyglądać jeśli wypiszę to samo, ale odpowiednio na więcej niż 7 miejscach i liczbie miejsc mniejszej niż 7.

> __[UWAGA]__ Każda wypisywana wartość musi mieć przypisany format.

In [118]:
awk -F ',' 'NR<5{printf("%1.4f\n", $2)}' TIC12974182-comma.dat    # liczba miejsc wskazana jako 1

-0.0655
-0.2563
0.0557
0.3494


> Podana została niewystarczająca liczba miejsc, więc nasze dane zostały wypisane na ich najmniejszej możliwej liczbie.

In [119]:
awk -F ',' 'NR<5{printf("%15.4f\n", $2)}' TIC12974182-comma.dat    # liczba miejsc wskazana jako 15

        -0.0655
        -0.2563
         0.0557
         0.3494


> Wszystko jest wyjustowane do prawej strony i dodatkowo, dodane zostały białe znaki w celu uzupełnienia 15 wskazanych miejsc.

|  Specyfikacja formatu  | * | * | * | W jaki sposób zostanie wypisane | 
| :-------------------:  | : | : | : | :-----------------------------: |
|         `%c`           | : | : | : |         pojedynczy znak         |
|         `%d`           | : | : | : |         liczba całkowita        |
|         `%e`           | : | : | : |       notacja inżynierska       |
|         `%f`           | : | : | : |        liczba rzeczywista       |
|         `%g`           | : | : | : |         automatyczny wybór pomiędzy `%f`, a `%e` (zależy co jest krótsze) i usunięcie zer (trailing zeros)        |
|         `%s`           | : | : | : |         ciąg znaków (string)        |
|         `%%`           | : | : | : |         aby wypisać znak `%`        |

We wcześniejszych przykładach widać liczby pomiędzy znakami `%` oraz literą formatu. Opisane zostało wypisywanie sformatowanej liczby rzeczywistej. Ogólny zapis jest taki: __`%[pierwsza_liczba.druga_liczba]format`__. Pierwsza liczba oznacza długość, druga w przypadku liczb rzeczywistych - precyzję, a w przypadku ciągu znaków - długość (podobnie jak w zmiennej znakowej z `bash`-a).<br> Przykłady:

In [132]:
awk 'BEGIN{printf("%5.3s\n", "abcdefg")}' # na pięciu miejscach wypisaną zostane 3 pierwsze znaki z ciągu `abcdefg`

  abc


In [126]:
awk -F ',' 'NR<5{printf("%15.4e\n", $2)}' TIC12974182-comma.dat    # tutaj przykład z notacją inżynierską

    -6.5509e-02
    -2.5634e-01
     5.5709e-02
     3.4936e-01


Zauważmy, że wszędzie na końcu formatu dopisany jest __znak nowej linii__ `\n`. Jeśli by go nie było, wszystko zostałoby wypisane w jednej linii:

In [130]:
awk -F ',' 'NR<5{printf("%15.4e", $2)}' TIC12974182-comma.dat    # wszystko w jednej linii

    -6.5509e-02    -2.5634e-01     5.5709e-02     3.4936e-01

Dodatkowe modyfikatory wpisywane po znaku `%`:

|      Modyfikator       | * | * | * | W jaki sposób zostanie wypisane | 
| :-------------------:  | : | : | : | :-----------------------------: |
|         `-`           | : | : | : |         justowanie do lewej strony         |
|         `<spacja>`           | : | : | : |         liczby dodatnie będą miały spację przed pierwszym znakiem, a ujemne `-`        |
|         `+`           | : | : | : |       nadpisuje modyfikator <spacja>, zawsze dodaje znak `+` przed liczbą dodatnią       |
|         `#`           | : | : | : |        forma alternatywna, np. nie usuwa nieznaczących zer (trailing zeros) dla `%g`       |
|         `0`           | : | : | : |         wypełnianie miejsc nie za pomocą białych znaków, ale przy pomocy `0`        |
|         `width`           | : | : | : |         width, to liczba (__pierwsza_liczba__) opisana wcześniej - długość        |
|         `.prec`           | : | : | : |         opisana wcześniej prezycja (__druga_liczba__) (dla liczb rzeczywistych) lub  ilość znaków dla string-ów i liczb całkowitych      |

## 6.4 Tablice asocjacyjne

AWK posiada możliwość tworzenia __tablic asocjacyjnych__, czyli takich gdzie indeksy nie muszą być sekwencją kolejnych liczb – mogą to być ciągi znaków lub liczby niekoniecznie uprządkowane numerycznie. Nazsa tych tablic jest samotłumacząca - asocjacja, czyli związek pomiędzy dwoma obiektami. Do danej wartości można się odnieść przez ten związek, czyli tablicę która wiąże ją z wskazaną przez nas nazwą. Dla przykładu, stworzę tablicę asocjacyjną, która może opisywać np. gwiazdę. Dla czytelności, można każdy wpis podzielić na wiersze i zakończyć średnikiem.

In [1]:
awk 'BEGIN{
gwiazda["nazwa_hd"]="HD204";
gwiazda["RA"]="01d47m16.35s";
gwiazda["DEC"]="72d46m30.76s";
printf("%30s %-20s\n", "Nazwa w katalogu HD: ", gwiazda["nazwa_hd"]);
printf("%30s %-20s\n", "Rekstascensja: ", gwiazda["RA"]);
printf("%30s %-20s\n", "Deklinacja: ", gwiazda["DEC"]);
}'

         Nazwa w katalogu HD:  HD204               
               Rekstascensja:  01d47m16.35s        
                  Deklinacja:  72d46m30.76s        


## 6.5 Część ciągu znaków `substr`

W `awk` możliwe jest również wypisanie części ciągu znaków, podobnie jak wpisywanie części zmiennej znakowej w `bash`-u (`${zmienna:offset:lenght}`). Służy do tego funkcja w `awk` o nazwie `substr`. Składnia jest następująca:
```bash
awk '{print substr(zmienna,offset,lenght)}'
```
Oznaczenia są takie same wcześniej.
Poniżej przykłady:

In [30]:
awk -F ',' 'NR>8529{print substr($1,5,4)}' TIC12974182-comma.dat   # Wypisanie 4 znaków zaczynając od 5 miejsca dla pierwszego pola (dla 3 ostatnich linii)

.167
.169
.170


In [48]:
awk -F ',' 'NR>8529{print "Część dziesiętna (3 cyfry) z czasu: "substr($1,5,4)"\nCzęść dziesiętna (4 cyfry) z błędu strumienia: " substr($3,2,5)"\n"}' TIC12974182-comma.dat

Część dziesiętna (3 cyfry) z czasu: .167
Część dziesiętna (4 cyfry) z błędu strumienia: .1972

Część dziesiętna (3 cyfry) z czasu: .169
Część dziesiętna (4 cyfry) z błędu strumienia: .1970

Część dziesiętna (3 cyfry) z czasu: .170
Część dziesiętna (4 cyfry) z błędu strumienia: .1973



> W powyższym przykładzie, gdy struktura pliku jest ustalona, można wypisać różne informacje. <br>W tym przypadku wypisane zostały (dla 3 ostatnich linii) części dzisiętne dla __pierwszej__ (`$1`) i __trzeciej__ (`$3`) kolumy z odpowiednio 3 i 4 cyframi po przecinku.